In [1]:
# Imports
import gymnasium as gym
from typing import Callable

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
import torch as th

import pickle

# Import Our environment
from dev_env import tradingEng


c:\Users\osc16\miniconda3\envs\d2d\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
# Load Paths
with open("1.6kRunDemo.pkl","rb") as fp:
    paths = pickle.load(fp)

In [3]:
## LR schedule
def linear_schedule(initial_value: float) -> Callable[[float], float]:
    """
    Linear learning rate schedule.

    :param initial_value: Initial learning rate.
    :return: schedule that computes
      current learning rate depending on remaining progress
    """
    def func(progress_remaining: float) -> float:
        """
        Progress will decrease from 1 (beginning) to 0.

        :param progress_remaining:
        :return: current learning rate
        """
        return progress_remaining * initial_value

    return func

In [4]:
# Def Env
env = tradingEng(paths, action = 'small', obs = 'xs')

# Instantiate the agent
policy_kwargs = dict(activation_fn=th.nn.ReLU,
                     net_arch=dict(pi=[512,256,128,64,36,18], vf=[512,256,128,64,36,18], optimizers_class = th.optim.Adam, log_std_init = False, ortho_init = False))
model = PPO("MlpPolicy", env, verbose=True, batch_size = 1024, learning_rate=linear_schedule(1e-4), policy_kwargs=policy_kwargs, n_steps=4096, normalize_advantage=True, vf_coef = 0.9, gamma = 0.9, ent_coef=0.005)

# Train the agent
#model = PPO.load("PPOsmall") # Resume training option
#model.set_env(env)
#model.policy_kwargs = dict(activation_fn=th.nn.ReLU,
#                     net_arch=dict(pi=[512,256,128,64,36,18], vf=[512,256,128,64,36,18], optimizers_class = th.optim.Adam))
#model.learning_rate = linear_schedule(1e-4)
#model.batch_size = 1024
#model.ent_coef = 0.00

model.learn(total_timesteps=int(3e6), progress_bar=False)

# Save the agent
model.save("PPOsmall")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 39.9     |
|    ep_rew_mean     | -2.44    |
| time/              |          |
|    fps             | 1070     |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 4096     |
---------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 39.6          |
|    ep_rew_mean          | -2.07         |
| time/                   |               |
|    fps                  | 943           |
|    iterations           | 2             |
|    time_elapsed         | 8             |
|    total_timesteps      | 8192          |
| train/                  |               |
|    approx_kl            | 2.7514383e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2       

In [6]:
import numpy as np

path = paths[2]
lambdax = path.lambdas[25]
r = path.r[25]
t = path.t_s[25]

model.predict(observation=np.asmatrix([lambdax,r]),state=None, deterministic=True)

(array([[-0.06065861, -0.02590458]]), None)